In [71]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import cv2
import mediapipe as mp
import os
import subprocess
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Prepare Data Testing Image 

In [72]:
def list_jpg_png_files(directory):
    jpg_png_files = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(root, filename)
                file_path = file_path.replace('\\', '/')
                jpg_png_files.append(file_path)
    return jpg_png_files

list_jpg_png_files('./datasets/Testing-Image')

['./datasets/Testing-Image/11_N_92.jpg',
 './datasets/Testing-Image/11_q_.jpg',
 './datasets/Testing-Image/1_ibu_.jpg',
 './datasets/Testing-Image/87_kerja_1.jpg',
 './datasets/Testing-Image/8_kerja_1.jpg',
 './datasets/Testing-Image/8_malam_1.jpg',
 './datasets/Testing-Image/8_malam_2.jpg',
 './datasets/Testing-Image/8_siang_2.jpg',
 './datasets/Testing-Image/91_jawab_1.jpg',
 './datasets/Testing-Image/91_kantor_1.jpg',
 './datasets/Testing-Image/92_jawab_1.jpg',
 './datasets/Testing-Image/92_kantor_1.jpg',
 './datasets/Testing-Image/93_jawab_1.jpg',
 './datasets/Testing-Image/93_sore_1.jpg',
 './datasets/Testing-Image/95_bagaimana_1.jpg',
 './datasets/Testing-Image/95_bagaimana_2.jpg',
 './datasets/Testing-Image/96_bagaimana_1.jpg',
 './datasets/Testing-Image/96_besok_1.jpg',
 './datasets/Testing-Image/96_kemarin_2.jpg',
 './datasets/Testing-Image/96_nanti_1.jpg',
 './datasets/Testing-Image/97_lusa_1.jpg',
 './datasets/Testing-Image/97_nanti_1.jpg',
 './datasets/Testing-Image/98_seka

# Test Static Image

In [73]:
def extract_hand_features(image_path):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5
    )

    data = []

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for landmark in hand_landmarks.landmark:
                landmarks.append(landmark.x)
                landmarks.append(landmark.y)
                landmarks.append(landmark.z)
            data.append([os.path.basename(image_path)] + landmarks)

    hands.close()

    return data


# sample_data = extract_hand_features("datasets/Testing-Image/0_Apa_1.jpg")[0][1:]
samples = []
samples_path = []
for file in list_jpg_png_files('./datasets/Testing-Image'):
    samples_path.append(file)
    samples += extract_hand_features(file)

c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [74]:
samples_path[:5]

['./datasets/Testing-Image/11_N_92.jpg',
 './datasets/Testing-Image/11_q_.jpg',
 './datasets/Testing-Image/1_ibu_.jpg',
 './datasets/Testing-Image/87_kerja_1.jpg',
 './datasets/Testing-Image/8_kerja_1.jpg']


# Test Predict

In [75]:
import joblib
import numpy as np

scaler = joblib.load("scaler.pkl")

for i in range(len(samples)):
    # # with scaler 
    samples[i] = samples[i][1:]
    samples[i] = np.array(samples[i]).reshape(1, -1)
    samples[i] = scaler.transform(samples[i])

    # without scaler
    # samples[i] =  np.expand_dims(samples[i][1:], axis=0)

label_encoder = joblib.load("label_encoder.pkl")

c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\

In [76]:
import tensorflow as tf

model_path = 'models/14-juni-1058am.h5'
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 64)                4096      
                                                                 
 dense_34 (Dense)            (None, 32)                2080      
                                                                 
 dense_35 (Dense)            (None, 66)                2178      
                                                                 
Total params: 8354 (32.63 KB)
Trainable params: 8354 (32.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [77]:
class_labels = ['A', 'Aku', 'Apa', 'B', 'Bagaimana', 'Baik', 'Bapak', 'Berapa',
    'Besok', 'C', 'D', 'Dia', 'Dimana', 'E', 'F', 'G', 'H', 'Halo',
    'Hari ini', 'I', 'Ibu', 'J', 'Jawab', 'K', 'Kalian', 'Kamu',
    'Kantor', 'Kapan', 'Kemana', 'Kemarin', 'Kerja', 'L', 'Lelah',
    'Lusa', 'M', 'Maaf', 'Makan', 'Malam', 'Mengapa', 'N', 'Nanti',
    'O', 'P', 'Pagi', 'Q', 'R', 'S', 'Sabar', 'Sakit', 'Sama - sama',
    'Sedih', 'Sekarang', 'Senang', 'Siang', 'Siapa', 'Sore', 'T',
    'Terima kasih', 'Tidur', 'Tolong', 'U', 'V', 'W', 'X', 'Y', 'Z']

def closest_label(input_word):
    global class_labels
    def levenshtein_distance(s1, s2):
        if len(s1) < len(s2):
            return levenshtein_distance(s2, s1)
        if len(s2) == 0:
            return len(s1)
        
        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row
        
        return previous_row[-1]
    
    closest_label = None
    min_distance = float('inf')
    
    for label in class_labels:
        if input_word.lower() in label.lower() and len(label) >= len(input_word):
            distance = levenshtein_distance(input_word.lower(), label.lower())
            if distance < min_distance:
                min_distance = distance
                closest_label = label
    
    return closest_label

In [78]:
# Contoh penggunaan
input_word = 'nanti'
closest = closest_label(input_word)
print(f"Kata '{input_word}' paling mendekati label: '{closest}'")

Kata 'nanti' paling mendekati label: 'Nanti'


In [79]:
"9_nanti_1".split('_')[1]

'nanti'

In [90]:
"kemarin" == "senang"

False

In [93]:
import numpy as np

log_entries = []
true = 0
false = 0

for i in range(len(samples)):
    print(f"predicted {samples_path[i]}")
    prediction = model.predict(samples[i])
    prediction = np.argmax(prediction, axis=1)
    prediction = class_labels[prediction[0]]
    
    label = samples_path[i].split('_')
    label = label[1]
    
    label = closest_label(label)
    
    status = None
    if label != None:
        if label == prediction:
            status = 'True'
            true += 1
        else:
            status = 'False'
            false += 1
    # else
    # Append log entry to the list
    log_entries.append({
        'samples_path': samples_path[i],
        'status': status,
        'true_value': label,
        'predicted_value': prediction
    })
    print(f'==> {status} | T:{label}|P:{prediction}')

print(f"Done Predict {true+false} data with {true} True and {false} False")            


predicted ./datasets/Testing-Image/11_N_92.jpg
1/1 [==============================] - 0s 16ms/step
==> True | T:N|P:N
predicted ./datasets/Testing-Image/11_q_.jpg
1/1 [==============================] - 0s 14ms/step
==> True | T:Q|P:Q
predicted ./datasets/Testing-Image/1_ibu_.jpg
1/1 [==============================] - 0s 16ms/step
==> True | T:Ibu|P:Ibu
predicted ./datasets/Testing-Image/87_kerja_1.jpg
1/1 [==============================] - 0s 15ms/step
==> True | T:Kerja|P:Kerja
predicted ./datasets/Testing-Image/8_kerja_1.jpg
1/1 [==============================] - 0s 15ms/step
==> True | T:Kerja|P:Kerja
predicted ./datasets/Testing-Image/8_malam_1.jpg
1/1 [==============================] - 0s 14ms/step
==> True | T:Malam|P:Malam
predicted ./datasets/Testing-Image/8_malam_2.jpg
1/1 [==============================] - 0s 13ms/step
==> True | T:Malam|P:Malam
predicted ./datasets/Testing-Image/8_siang_2.jpg
1/1 [==============================] - 0s 17ms/step
==> True | T:Siang|P:Siang
pred

In [94]:
df_logs = pd.DataFrame(log_entries)
df_logs.head(100)

,samples_path,status,true_value,predicted_value
0,./datasets/Testing-Image/11_N_92.jpg,True,N,N
1,./datasets/Testing-Image/11_q_.jpg,True,Q,Q
2,./datasets/Testing-Image/1_ibu_.jpg,True,Ibu,Ibu
3,./datasets/Testing-Image/87_kerja_1.jpg,True,Kerja,Kerja
4,./datasets/Testing-Image/8_kerja_1.jpg,True,Kerja,Kerja
5,./datasets/Testing-Image/8_malam_1.jpg,True,Malam,Malam
6,./datasets/Testing-Image/8_malam_2.jpg,True,Malam,Malam
7,./datasets/Testing-Image/8_siang_2.jpg,True,Siang,Siang
8,./datasets/Testing-Image/91_jawab_1.jpg,True,Jawab,Jawab
9,./datasets/Testing-Image/91_kantor_1.jpg,True,Kantor,Kantor


: 